In [2]:
import torch
import numpy as np
from torch import nn
from utils.model import Spice_model
from torchinfo import summary
import pandas as pd
from data_files.dataloader import MedleyDBLoader

In [3]:
fs = 16000
med = MedleyDBLoader(fs )

In [4]:
med.get_ids()

['MusicDelta_Country1_STEM_05',
 'MusicDelta_Pachelbel_STEM_04',
 'MusicDelta_Disco_STEM_04',
 'MusicDelta_ChineseYaoZu_STEM_01',
 'MusicDelta_Country2_STEM_05',
 'BigTroubles_Phantom_STEM_04',
 'MusicDelta_CoolJazz_STEM_05',
 'MusicDelta_SwingJazz_STEM_04',
 'HeladoNegro_MitadDelMundo_STEM_08',
 'ClaraBerryAndWooldog_Stella_STEM_07',
 'AvaLuna_Waterduct_STEM_08',
 'SecretMountains_HighHorse_STEM_09',
 'MusicDelta_BebopJazz_STEM_05',
 'MusicDelta_Rock_STEM_05',
 'PortStWillow_StayEven_STEM_08',
 'MusicDelta_Beatles_STEM_07',
 'MusicDelta_Grunge_STEM_05',
 'MusicDelta_FreeJazz_STEM_03',
 'AClassicEducation_NightOwl_STEM_08',
 'ClaraBerryAndWooldog_WaltzForMyVictims_STEM_05',
 'MatthewEntwistle_FairerHopes_STEM_14',
 'AimeeNorwich_Child_STEM_04',
 'MusicDelta_ModalJazz_STEM_05',
 'MusicDelta_FusionJazz_STEM_06',
 'AlexanderRoss_GoodbyeBolero_STEM_06',
 'TheDistricts_Vermont_STEM_05',
 'Meaxic_TakeAStep_STEM_04',
 'StrandOfOaks_Spacestation_STEM_04',
 'MusicDelta_80sRock_STEM_04',
 'Matth

In [8]:
f0[2]

array([0., 0., 0., ..., 0., 0., 0.])

In [16]:
a = np.arange(0,15,1).reshape(5,3)
a = np.hstack((a, np.array([1,0,0,1,0]).reshape(-1,1)))
a

array([[ 0,  1,  2,  1],
       [ 3,  4,  5,  0],
       [ 6,  7,  8,  0],
       [ 9, 10, 11,  1],
       [12, 13, 14,  0]])

In [62]:
df = pd.DataFrame(a)
df

,0,1,2,3
0,0,1,2,1
1,3,4,5,0
2,6,7,8,0
3,9,10,11,1
4,12,13,14,0


In [63]:
# remove rows of cqt where label (last) column is zero
df.drop(df.loc[df.iloc[:, -1]==0].index, inplace=True)
df

,0,1,2,3
0,0,1,2,1
3,9,10,11,1


In [9]:
s = Spice_model()
summary(s, input_size=[10, 128])

no unpooling True True
no unpooling False True
no unpooling False True
no unpooling False True
no unpooling False True


Layer (type:depth-idx)                   Output Shape              Param #
Spice_model                              [10, 1]                   --
├─Spice_Encoder: 1-1                     [10, 1]                   --
│    └─Conv_block: 2-1                   [10, 64, 64]              --
│    │    └─Conv1d: 3-1                  [10, 64, 128]             256
│    │    └─BatchNorm1d: 3-2             [10, 64, 128]             128
│    │    └─ReLU: 3-3                    [10, 64, 128]             --
│    │    └─MaxPool1d: 3-4               [10, 64, 64]              --
│    └─Conv_block: 2-2                   [10, 128, 32]             --
│    │    └─Conv1d: 3-5                  [10, 128, 64]             24,704
│    │    └─BatchNorm1d: 3-6             [10, 128, 64]             256
│    │    └─ReLU: 3-7                    [10, 128, 64]             --
│    │    └─MaxPool1d: 3-8               [10, 128, 32]             --
│    └─Conv_block: 2-3                   [10, 256, 16]             --
│    │  